In [ ]:
!pip install --upgrade pip setuptools wheel
!pip install pyarrow==9.0.0
!pip install datasets



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 81.6 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 20.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 9.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.3 MB/s eta 0:00:00
  A

In [ ]:

!pip install transformers


In [ ]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 35.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2


In [ ]:
import os
import json
from PIL import Image
from transformers import LayoutXLMProcessor
import torch
from transformers import LayoutLMTokenizerFast, LayoutLMv2FeatureExtractor, LayoutLMForTokenClassification
from transformers import LayoutXLMTokenizerFast, LayoutLMv2Processor, LayoutXLMProcessor, LayoutXLMTokenizer
from datasets import Dataset, DatasetDict, Features, Sequence, ClassLabel, Value, Array2D, Array3D
from transformers import LayoutLMForTokenClassification, AdamW
from tqdm.notebook import tqdm

**Load data**

In [ ]:
from google.colab import files
import zipfile
uploaded = files.upload()

zip_file_name = next(iter(uploaded))



Saving ia.zip to ia.zip


In [ ]:
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall('/content/folder')

# Définir le chemin du sous-dossier contenant les images et les fichiers JSON
folder_path = '/content/folder'

files = os.listdir(folder_path)

image_paths = [os.path.join(folder_path, f) for f in files if f.lower().endswith(('.jpg', '.png'))]
json_paths = [os.path.join(folder_path, f) for f in files if f.endswith('.json')]

assert len(image_paths) == len(json_paths), "Le nombre d'images et de fichiers JSON ne correspond pas"

print(f"Images trouvées: {image_paths}")
print(f"Fichiers JSON trouvés: {json_paths}")


Images trouvées: ['/content/folder/DEVIS SAFRAN SEAT_T056533_1_image_0.jpg', '/content/folder/Devis N° 24QI00131-Table de contrôle+Chario_T056570_2_image_0.jpg', '/content/folder/Devis_302000000131_T057038_1_image_0.jpg', '/content/folder/Devis cde_241540_T057111_1_image_0.jpg', '/content/folder/devis 81_T057169_1_image_0.jpg', '/content/folder/Devis_40793_DE2401298_2024042_T056506_1_image_1.jpg', '/content/folder/Devis_302000000131_T057038_1_image_1.jpg', '/content/folder/FB Devis (050_T056877_1_image_0.jpg', '/content/folder/DEVIS 6_T056399__T056873_1_image_0.jpg', '/content/folder/TT24-085_T057028_1_image_0.jpg', '/content/folder/QUOTE SAFRAN 101912894815_T057199_1_image_1.jpg', '/content/folder/ZTP Devis Jawher lame de machine dénudag_T056774_1_image_0.jpg', '/content/folder/DEVME00743_T056727_1_image_1.jpg', '/content/folder/T05673_T056731_1_image_0.jpg', '/content/folder/devis 3_T056580_1_image_0.jpg', '/content/folder/devis N° 103_20240425_000_T056474_1_image_0.jpg', '/content/f

**Preprocess data**

In [ ]:


def preprocess_data(image_path, json_path, label_map):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    image = Image.open(image_path).convert("RGB")

    words = []
    boxes = []
    labels = []

    for item in data["form"]:
        for word in item["words"]:
            words.append(word["text"])
            boxes.append(word["box"])
            label = item.get("label", "O")
            labels.append(label_map.get(label, label_map["O"]))

    width, height = image.size
    normalized_boxes = [
        [
            int(1000 * (box[0] / width)),
            int(1000 * (box[1] / height)),
            int(1000 * (box[2] / width)),
            int(1000 * (box[3] / height))
        ]
        for box in boxes
    ]

    return words, normalized_boxes, labels

def process_folder(folder_path, label_map):
    data = {
        'id': [],
        'words': [],
        'bboxes': [],
        'ner_tags': [],
        'image_path': []
    }

    for file_name in os.listdir(folder_path):
        if file_name.endswith('.json'):
            json_path = os.path.join(folder_path, file_name)
            image_path = os.path.join(folder_path, file_name.replace('.json', '.png'))  # Assuming images are .png
            if not os.path.exists(image_path):
                image_path = os.path.join(folder_path, file_name.replace('.json', '.jpg'))  # Trying with .jpg extension

            if os.path.exists(image_path):
                words, boxes, labels = preprocess_data(image_path, json_path, label_map)
                data['id'].append(file_name.replace('.json', ''))
                data['words'].append(words)
                data['bboxes'].append(boxes)
                data['ner_tags'].append(labels)
                data['image_path'].append(image_path)
            else:
                print(f"Image corresponding to {json_path} not found.")

    return data


all_labels = ["NOM_FOURNISSEUR", "NUMERO_DE_DEVIS", "DESIGNATION", "QUANTITE", "PRIX", "TOTAL_NET", "MATRICULE_FISCALE", "TIMBRE_FISCALE", "REMISE", "FRAIS_DE_TRANSPORT", "O"]
label_map = {label: idx for idx, label in enumerate(all_labels)}


tokenizer = LayoutXLMTokenizerFast.from_pretrained("microsoft/layoutxlm-base")
feature_extractor = LayoutLMv2Processor.from_pretrained("microsoft/layoutlmv2-base-uncased")
folder_path = "/content/folder"
data = process_folder(folder_path, label_map)

dataset = Dataset.from_dict(data)

train_test_split = dataset.train_test_split(test_size=0.25)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

print(dataset_dict)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

config.4.13.0.json:   0%|          | 0.00/947 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/135 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'words', 'bboxes', 'ner_tags', 'image_path'],
        num_rows: 278
    })
    test: Dataset({
        features: ['id', 'words', 'bboxes', 'ner_tags', 'image_path'],
        num_rows: 93
    })
})


In [ ]:

labels = ["NOM_FOURNISSEUR", "NUMERO_DE_DEVIS", "DESIGNATION", "QUANTITE", "PRIX", "TOTAL_NET", "MATRICULE_FISCALE", "TIMBRE_FISCALE", "REMISE", "FRAIS_DE_TRANSPORT"]
print(labels)
id2label = {v: k for v, k in enumerate(labels)}
label2id = {k: v for v, k in enumerate(labels)}
print("id2label:", id2label)
print("label2id:", label2id)


['NOM_FOURNISSEUR', 'NUMERO_DE_DEVIS', 'DESIGNATION', 'QUANTITE', 'PRIX', 'TOTAL_NET', 'MATRICULE_FISCALE', 'TIMBRE_FISCALE', 'REMISE', 'FRAIS_DE_TRANSPORT']
id2label: {0: 'NOM_FOURNISSEUR', 1: 'NUMERO_DE_DEVIS', 2: 'DESIGNATION', 3: 'QUANTITE', 4: 'PRIX', 5: 'TOTAL_NET', 6: 'MATRICULE_FISCALE', 7: 'TIMBRE_FISCALE', 8: 'REMISE', 9: 'FRAIS_DE_TRANSPORT'}
label2id: {'NOM_FOURNISSEUR': 0, 'NUMERO_DE_DEVIS': 1, 'DESIGNATION': 2, 'QUANTITE': 3, 'PRIX': 4, 'TOTAL_NET': 5, 'MATRICULE_FISCALE': 6, 'TIMBRE_FISCALE': 7, 'REMISE': 8, 'FRAIS_DE_TRANSPORT': 9}


**1:Initialisation du processeur et des caractéristiques**

**2. Définition de la fonction de prétraitement**

**3. Application de la fonction de prétraitement aux datasets d'entraînement et de test**

In [ ]:
labels = ["NOM_FOURNISSEUR", "NUMERO_DE_DEVIS", "DESIGNATION", "QUANTITE", "PRIX", "TOTAL_NET", "MATRICULE_FISCALE", "TIMBRE_FISCALE", "REMISE", "FRAIS_DE_TRANSPORT"]


label_map = {label: idx for idx, label in enumerate(labels)}

processor = LayoutXLMProcessor.from_pretrained("microsoft/layoutxlm-base", apply_ocr=False)

features = Features({
    'image': Array3D(dtype="int64", shape=(3, 224, 224)),
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'attention_mask': Sequence(Value(dtype='int64')),
    'token_type_ids': Sequence(Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'labels': Sequence(ClassLabel(names=labels)),
})

def preprocess_data(examples):
    images = [Image.open(path).convert("RGB") for path in examples['image_path']]
    words = examples['words']
    boxes = examples['bboxes']
    word_labels = examples['ner_tags']

    encoded_inputs = processor(images, words, boxes=boxes, word_labels=word_labels,
                               padding="max_length", truncation=True, max_length=512, return_token_type_ids=True)

    return encoded_inputs

def process_folder(folder_path, label_map):
    data = {
        'id': [],
        'words': [],
        'bboxes': [],
        'ner_tags': [],
        'image_path': []
    }

    for file_name in os.listdir(folder_path):
        if file_name.endswith('.json'):
            json_path = os.path.join(folder_path, file_name)
            image_path = os.path.join(folder_path, file_name.replace('.json', '.png'))  # Assuming images are .png
            if not os.path.exists(image_path):
                image_path = os.path.join(folder_path, file_name.replace('.json', '.jpg'))  # Trying with .jpg extension

            if os.path.exists(image_path):
                with open(json_path, 'r', encoding='utf-8') as f:
                    data_json = json.load(f)

                words, boxes, labels = [], [], []
                for item in data_json["form"]:
                    for word in item["words"]:
                        words.append(word["text"])
                        boxes.append(word["box"])
                        label = item.get("label")
                        if label in label_map:  # Only add if the label is in the label_map (excluding "O")
                            labels.append(label_map[label])

                width, height = Image.open(image_path).size
                normalized_boxes = [
                    [
                        int(1000 * (box[0] / width)),
                        int(1000 * (box[1] / height)),
                        int(1000 * (box[2] / width)),
                        int(1000 * (box[3] / height))
                    ]
                    for box in boxes
                ]

                data['id'].append(file_name.replace('.json', ''))
                data['words'].append(words)
                data['bboxes'].append(normalized_boxes)
                data['ner_tags'].append(labels)
                data['image_path'].append(image_path)
            else:
                print(f"Image corresponding to {json_path} not found.")

    return data

# Exemple d'utilisation du dossier
folder_path = "/content/folder"  # Remplacez par le chemin de votre dossier
data = process_folder(folder_path, label_map)

# Conversion en Dataset Hugging Face
dataset = Dataset.from_dict(data)

# Division en ensemble d'entraînement et de test
train_test_split = dataset.train_test_split(test_size=0.25)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

# Création d'un DatasetDict
datasets = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

# Prétraitement des données d'entraînement et de test
train_dataset = datasets['train'].map(
    preprocess_data,
    batched=True,
    remove_columns=datasets['train'].column_names,
    features=features
)

test_dataset = datasets['test'].map(
    preprocess_data,
    batched=True,
    remove_columns=datasets['test'].column_names,
    features=features
)

print(train_dataset)
print(test_dataset)



preprocessor_config.json:   0%|          | 0.00/135 [00:00<?, ?B/s]

Map:   0%|          | 0/278 [00:00<?, ? examples/s]

Map:   0%|          | 0/93 [00:00<?, ? examples/s]

Dataset({
    features: ['image', 'input_ids', 'attention_mask', 'token_type_ids', 'bbox', 'labels'],
    num_rows: 278
})
Dataset({
    features: ['image', 'input_ids', 'attention_mask', 'token_type_ids', 'bbox', 'labels'],
    num_rows: 93
})


In [ ]:
tokenizer = LayoutXLMTokenizer.from_pretrained("microsoft/layoutxlm-base")

# Récupérez les input_ids de votre dataset
input_ids = train_dataset['input_ids'][1]

# Décodez les input_ids en texte
decoded_text = tokenizer.decode(input_ids, skip_special_tokens=True)

print(decoded_text)


DMS TECHNOLOGIES 950373 P / A / M / 000 240262 CONCEPTION ET CONFECTION CALE 260 * 20 * 10MM EN 2 10 148,500 POM - C POUR OUTIL DE SABLAGE 297,000 1,000


In [ ]:
print(train_dataset['labels'][1])

[-100, 0, -100, 0, 0, -100, -100, -100, 6, -100, -100, 6, 6, 6, 6, 6, 6, 6, 1, -100, -100, 2, -100, -100, 2, 2, -100, -100, -100, 2, -100, 2, 2, 2, 2, 2, -100, 2, 3, 8, 4, -100, -100, 2, -100, 2, 2, 2, -100, 2, 2, -100, 2, 2, -100, -100, 5, -100, 7, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 

In [ ]:
train_dataset.set_format(type="torch", device="cuda")
test_dataset.set_format(type="torch", device="cuda")


In [ ]:
train_dataset.features.keys()

dict_keys(['image', 'input_ids', 'attention_mask', 'token_type_ids', 'bbox', 'labels'])

In [ ]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=2)


In [ ]:
batch = next(iter(train_dataloader))

for k,v in batch.items():
  print(k, v.shape)

image torch.Size([4, 3, 224, 224])
input_ids torch.Size([4, 512])
attention_mask torch.Size([4, 512])
token_type_ids torch.Size([4, 512])
bbox torch.Size([4, 512, 4])
labels torch.Size([4, 512])


**Train the model**

In [ ]:
labels = ["NOM_FOURNISSEUR", "NUMERO_DE_DEVIS", "DESIGNATION", "QUANTITE", "PRIX", "TOTAL_NET", "MATRICULE_FISCALE", "TIMBRE_FISCALE", "REMISE", "FRAIS_DE_TRANSPORT"]
model = LayoutLMForTokenClassification.from_pretrained('microsoft/layoutxlm-base', num_labels=len(labels))

device ='cpu'
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)

global_step = 0
num_train_epochs = 6
model.train()

for epoch in range(num_train_epochs):
    print(f"Epoch: {epoch}")
    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()
        inputs = {
            'input_ids': batch['input_ids'].to(device),
            'attention_mask': batch['attention_mask'].to(device),
            'bbox': batch['bbox'].to(device),
            'labels': batch['labels'].to(device)
        }

        outputs = model(**inputs)
        loss = outputs.loss

        if global_step % 100 == 0:
            print(f"Loss after {global_step} steps: {loss.item()}")

        loss.backward()
        optimizer.step()
        global_step += 1


You are using a model of type layoutlmv2 to instantiate a model of type layoutlm. This is not supported for all configurations of models and can yield errors.


pytorch_model.bin:   0%|          | 0.00/1.48G [00:00<?, ?B/s]

Some weights of LayoutLMForTokenClassification were not initialized from the model checkpoint at microsoft/layoutxlm-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'layoutlm.embeddings.LayerNorm.bias', 'layoutlm.embeddings.LayerNorm.weight', 'layoutlm.embeddings.h_position_embeddings.weight', 'layoutlm.embeddings.position_embeddings.weight', 'layoutlm.embeddings.token_type_embeddings.weight', 'layoutlm.embeddings.w_position_embeddings.weight', 'layoutlm.embeddings.word_embeddings.weight', 'layoutlm.embeddings.x_position_embeddings.weight', 'layoutlm.embeddings.y_position_embeddings.weight', 'layoutlm.encoder.layer.0.attention.output.LayerNorm.bias', 'layoutlm.encoder.layer.0.attention.output.LayerNorm.weight', 'layoutlm.encoder.layer.0.attention.output.dense.bias', 'layoutlm.encoder.layer.0.attention.output.dense.weight', 'layoutlm.encoder.layer.0.attention.self.key.bias', 'layoutlm.encoder.layer.0.attention.self.key.weight', 'layoutlm.encoder.layer.0.attentio

Epoch: 0


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

Epoch: 4


  0%|          | 0/70 [00:00<?, ?it/s]

Loss after 300 steps: 0.010155082680284977
Epoch: 5


  0%|          | 0/70 [00:00<?, ?it/s]

Loss after 400 steps: 0.0068468512035906315


**Evaluate the model**

In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=04e2758169095dfda17954f546f0bd95df168b55bd763eac702a87ef66067b37
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
from datasets import load_metric

# Charger la métrique seqeval
metric = load_metric("seqeval", trust_remote_code=True)

# Mettre le modèle en mode évaluation
model.eval()

# Mapper les labels à leurs indices
id2label = {v: k for k, v in label_map.items()}

# Fonction pour inverser la normalisation des boîtes
def unnormalize_box(bbox, width, height):
    return [
        width * (bbox[0] / 1000),
        height * (bbox[1] / 1000),
        width * (bbox[2] / 1000),
        height * (bbox[3] / 1000),
    ]

# Évaluer le modèle sur les données de test
for batch in tqdm(test_dataloader, desc="Evaluating"):
    with torch.no_grad():
        input_ids = batch['input_ids'].to(device)
        bbox = batch['bbox'].to(device)
        image = batch['image'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, bbox=bbox, attention_mask=attention_mask, token_type_ids=token_type_ids, labels=labels)
        # Prédictions
        predictions = outputs.logits.argmax(dim=2)
        # Remove ignored index (special tokens)
        true_predictions = [
            [id2label[p.item()] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        true_labels = [
            [id2label[l.item()] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        # Add batch to metric
        metric.add_batch(predictions=true_predictions, references=true_labels)
# Calculer les résultats
results = metric.compute()

# Afficher les résultats
for label, metrics in results.items():
    if isinstance(metrics, dict):  # Vérifie si metrics est un dictionnaire
        label_corrected = label.replace("_", " ").capitalize()
        print(f"{label_corrected}:")
        print(f"  Precision: {metrics['precision']:.4f}")
        print(f"  Recall: {metrics['recall']:.4f}")
        print(f"  F1 Score: {metrics['f1']:.4f}")
        print(f"  Support: {metrics['number']}\n")
    else:
        print(f"{label}: {metrics}\n")


Evaluating: 100%|██████████| 47/47 [01:36<00:00,  2.04s/it]

Atricule fiscale:
  Precision: 0.4624
  Recall: 0.6935
  F1 Score: 0.5548
  Support: 62

Emise:
  Precision: 0.5309
  Recall: 0.6232
  F1 Score: 0.5733
  Support: 69

Esignation:
  Precision: 0.4846
  Recall: 0.7040
  F1 Score: 0.5740
  Support: 223

Imbre fiscale:
  Precision: 0.7917
  Recall: 0.7600
  F1 Score: 0.7755
  Support: 25

Om fournisseur:
  Precision: 0.6733
  Recall: 0.7727
  F1 Score: 0.7196
  Support: 88

Otal net:
  Precision: 0.6184
  Recall: 0.6026
  F1 Score: 0.6104
  Support: 78

Rais de transport:
  Precision: 0.9000
  Recall: 1.0000
  F1 Score: 0.9474
  Support: 9

Rix:
  Precision: 0.6437
  Recall: 0.6667
  F1 Score: 0.6550
  Support: 252

Uantite:
  Precision: 0.7016
  Recall: 0.7160
  F1 Score: 0.7088
  Support: 243

Umero de devis:
  Precision: 0.7778
  Recall: 0.7590
  F1 Score: 0.7683
  Support: 83

overall_precision: 0.6089299461123941

overall_recall: 0.6987632508833922

overall_f1: 0.6507610037021802

overall_accuracy: 0.9060183166157871




/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: TOTAL_NET seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: FRAIS_DE_TRANSPORT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: MATRICULE_FISCALE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NUMERO_DE_DEVIS seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NOM_FOURNISSEUR seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10

In [ ]:
example = datasets["test"][0]
print(example.keys())


dict_keys(['id', 'words', 'bboxes', 'ner_tags', 'image_path'])


In [ ]:
encoded_inputs = processor(image, example['words'], boxes=example['bboxes'], word_labels=example['ner_tags'],
                           padding="max_length", truncation=True, return_tensors="pt")

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
labels = encoded_inputs.pop('labels').squeeze().tolist()
for k,v in encoded_inputs.items():
  encoded_inputs[k] = v.to(device)

In [ ]:
# Remove the 'image' key from the encoded_inputs dictionary
encoded_inputs.pop('image', None)

# forward pass
outputs = model(**encoded_inputs)

In [ ]:
outputs.logits.shape

torch.Size([1, 15, 10])

In [ ]:
from PIL import Image, ImageDraw, ImageFont

# Fonction pour inverser la normalisation des boîtes englobantes
def unnormalize_box(bbox, width, height):
    return [
        width * (bbox[0] / 1000),
        height * (bbox[1] / 1000),
        width * (bbox[2] / 1000),
        height * (bbox[3] / 1000),
    ]

# Charger une image à partir de son chemin (assurez-vous que `image_path` est défini)
image_path = example['image_path']  # À adapter selon votre projet
image = Image.open(image_path)
image = image.convert("RGB")

# Récupérer les prédictions et les boîtes englobantes à partir des sorties du modèle (à adapter)
predictions = outputs.logits.argmax(-1).squeeze().tolist()
token_boxes = encoded_inputs.bbox.squeeze().tolist()  # Assurez-vous que `encoded_inputs.bbox` est correctement défini

# Dimensions de l'image
width, height = image.size

# Filtrer les prédictions et les boîtes englobantes valides
true_predictions = [id2label[prediction] for prediction, label in zip(predictions, labels) if label != -100]
true_boxes = [unnormalize_box(box, width, height) for box, label in zip(token_boxes, labels) if label != -100]

# Créer un objet ImageDraw pour annoter l'image
draw = ImageDraw.Draw(image)

# Définir une police pour les annotations
font = ImageFont.load_default()  # À adapter selon vos préférences

# Parcourir les prédictions et les boîtes englobantes pour annoter l'image
for prediction, bbox in zip(true_predictions, true_boxes):
    # Convertir les coordonnées de la boîte englobante en entiers
    bbox = [int(coord) for coord in bbox]
    # Dessiner un rectangle autour de la région d'intérêt
    draw.rectangle(bbox, outline="red")
    # Afficher le libellé au-dessus du rectangle
    draw.text((bbox[0], bbox[1] - 10), prediction, fill="red", font=font)

image.show()



In [ ]:
print(true_predictions)

['TOTAL_NET', 'TOTAL_NET', 'TOTAL_NET', 'FRAIS_DE_TRANSPORT', 'FRAIS_DE_TRANSPORT', 'MATRICULE_FISCALE']


In [ ]:
print(true_labels)

[['NOM_FOURNISSEUR', 'NOM_FOURNISSEUR', 'NOM_FOURNISSEUR', 'NOM_FOURNISSEUR', 'NOM_FOURNISSEUR', 'NOM_FOURNISSEUR', 'NOM_FOURNISSEUR', 'NOM_FOURNISSEUR', 'NOM_FOURNISSEUR', 'NOM_FOURNISSEUR', 'NOM_FOURNISSEUR', 'NOM_FOURNISSEUR', 'NOM_FOURNISSEUR', 'NOM_FOURNISSEUR', 'NOM_FOURNISSEUR', 'NOM_FOURNISSEUR', 'NOM_FOURNISSEUR', 'NOM_FOURNISSEUR', 'NOM_FOURNISSEUR', 'NOM_FOURNISSEUR', 'NOM_FOURNISSEUR', 'NOM_FOURNISSEUR', 'NUMERO_DE_DEVIS', 'DESIGNATION', 'DESIGNATION', 'DESIGNATION', 'DESIGNATION', 'DESIGNATION', 'DESIGNATION', 'DESIGNATION', 'DESIGNATION', 'DESIGNATION', 'DESIGNATION', 'QUANTITE', 'PRIX', 'TOTAL_NET', 'MATRICULE_FISCALE', 'MATRICULE_FISCALE', 'MATRICULE_FISCALE', 'MATRICULE_FISCALE']]


In [ ]:
print(true_boxes)

[[1056.417, 1101.336, 1063.563, 1116.492], [1071.9, 1101.336, 1129.068, 1118.1760000000002], [1071.9, 1101.336, 1129.068, 1118.1760000000002], [1092.147, 1138.384, 1127.877, 1151.856], [1092.147, 1138.384, 1127.877, 1151.856], [458.535, 1579.5919999999999, 574.062, 1593.0639999999999]]
